### Load libaries and files

In [1]:
import pickle
import pandas as pd
import json
import langdetect
import re
import Algorithmia
import bs4 as bs
import lxml
from urllib.request import Request, urlopen
import time
from fuzzywuzzy import process
import tfidf_matcher as tm
from fake_useragent import UserAgent

C:\Users\ACER\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
with open('../Data/pickled_df.pkl', 'rb') as f:
    data = pickle.load(f)
data = data.reset_index(drop = True)
muckrack_sitemap = pd.read_csv('../Data/muckrack_persons_fetchlist.csv')

In [3]:
data

,author,crawl_date,crawl_desc,crawl_url,date,description,full_text,img_url,site_name,title,topic,url,webpage_source,lang
0,"[""Liz Josaitis, Michigan State University Exte...",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,The Food and Drug Administration has offered a...,A few examples include the offering of anti-a...,https://www.westernjournal.com/wp-content/uplo...,The Western Journal,"Health Scammers Prey on the Elderly, Here's Ho...",politics,https://www.westernjournal.com/health-scammers...,westernjournal.com,en
1,"[""Aja Styles""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,https://static.ffx.io/images/$zoom_0.3431%2C$m...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...,brisbanetimes.com.au,en
2,"[""Jake Johnson""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,https://truthout.org/wp-content/uploads/2020/1...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...,truthout.org,en
3,"[""Christine Favocci""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,It is naive to think that either party is free...,The left has insisted that voter fraud is jus...,https://www.westernjournal.com/wp-content/uplo...,The Western Journal,PA Man Facing Charges of Unlawful Voting After...,tech,https://www.westernjournal.com/pa-man-facing-c...,westernjournal.com,en
4,"[""Igor Derysh""]",2020-12-23,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201223000...,2020-12-23,Fox aired segments debunking the claims after ...,Fox News and Fox Business just aired multiple...,https://truthout.org/wp-content/uploads/2020/1...,Truthout,Fox News Forced to Debunk Its Own False Voting...,business,https://truthout.org/articles/fox-news-forced-...,truthout.org,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267705,"[""Vanguard""]",2020-12-05,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201205052...,2020-12-05,"“There has never been a solid, unified and det...",The challenge of achieving racial justice in ...,https://i0.wp.com/www.vanguardngr.com/wp-conte...,Vanguard News,The challenge of achieving racial justice in A...,entertainment,https://www.vanguardngr.com/2020/12/the-challe...,vanguardngr.com,en
267706,"[""Vanguard""]",2020-12-05,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201205052...,2020-12-05,The United States on Friday imposed new sancti...,US imposes visa restrictions on Chinese offic...,https://i2.wp.com/www.vanguardngr.com/wp-conte...,Vanguard News,US imposes visa restrictions on Chinese offici...,business,https://www.vanguardngr.com/2020/12/us-imposes...,vanguardngr.com,en
267707,"[""Guto Llewelyn""]",2020-12-05,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201205052...,2020-12-05,Fan columnist Guto Llewelyn examines the situa...,When you subscribe we will use the informatio...,https://i2-prod.walesonline.co.uk/incoming/art...,WalesOnline,The reality of Swansea City's striker concerns...,sport,https://www.walesonline.co.uk/sport/football/t...,walesonline.co.uk,en
267708,"[""Laura McAllister""]",2020-12-05,news cc-crawl,crawl-data/CC-NEWS/2020/12/CC-NEWS-20201205052...,2020-12-05,"If successful, I’d be the first woman from the...",Former Wales international Laura McAllister h...,https://i2-prod.walesonline.co.uk/incoming/art...,WalesOnline,"FIFA, football and Wales: Why I am standing fo...",sport,https://www.walesonline.co.uk/sport/sport-opin...,walesonline.co.uk,en


### Initial preprocessing

#### Remove unnecessary columns

In [4]:
data = data.drop(['crawl_date', 'crawl_desc', 'crawl_url', 'img_url', 'webpage_source', 'lang'], 1)

In [5]:
data

,author,date,description,full_text,site_name,title,topic,url
0,"[""Liz Josaitis, Michigan State University Exte...",2020-12-23,The Food and Drug Administration has offered a...,A few examples include the offering of anti-a...,The Western Journal,"Health Scammers Prey on the Elderly, Here's Ho...",politics,https://www.westernjournal.com/health-scammers...
1,"[""Aja Styles""]",2020-12-23,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...
2,"[""Jake Johnson""]",2020-12-23,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...
3,"[""Christine Favocci""]",2020-12-23,It is naive to think that either party is free...,The left has insisted that voter fraud is jus...,The Western Journal,PA Man Facing Charges of Unlawful Voting After...,tech,https://www.westernjournal.com/pa-man-facing-c...
4,"[""Igor Derysh""]",2020-12-23,Fox aired segments debunking the claims after ...,Fox News and Fox Business just aired multiple...,Truthout,Fox News Forced to Debunk Its Own False Voting...,business,https://truthout.org/articles/fox-news-forced-...
...,...,...,...,...,...,...,...,...
267705,"[""Vanguard""]",2020-12-05,"“There has never been a solid, unified and det...",The challenge of achieving racial justice in ...,Vanguard News,The challenge of achieving racial justice in A...,entertainment,https://www.vanguardngr.com/2020/12/the-challe...
267706,"[""Vanguard""]",2020-12-05,The United States on Friday imposed new sancti...,US imposes visa restrictions on Chinese offic...,Vanguard News,US imposes visa restrictions on Chinese offici...,business,https://www.vanguardngr.com/2020/12/us-imposes...
267707,"[""Guto Llewelyn""]",2020-12-05,Fan columnist Guto Llewelyn examines the situa...,When you subscribe we will use the informatio...,WalesOnline,The reality of Swansea City's striker concerns...,sport,https://www.walesonline.co.uk/sport/football/t...
267708,"[""Laura McAllister""]",2020-12-05,"If successful, I’d be the first woman from the...",Former Wales international Laura McAllister h...,WalesOnline,"FIFA, football and Wales: Why I am standing fo...",sport,https://www.walesonline.co.uk/sport/sport-opin...


#### Convert lists to individual journalists

In [6]:
articles = []
for j in range(len(data)):
    try:
        contributors = json.loads(data.iloc[j].author)
        for contributor in contributors:
            articles.append([contributor, data.iloc[j].date, data.iloc[j].description, data.iloc[j].full_text, 
                         data.iloc[j].site_name, data.iloc[j].title, data.iloc[j].topic, data.iloc[j].url])
    except:
        pass

In [7]:
articles = pd.DataFrame(articles)
articles.columns = ['author', 'date', 'description', 'full_text', 'site_name', 'title', 'topic', 'url']

In [8]:
articles

,author,date,description,full_text,site_name,title,topic,url
0,"Liz Josaitis, Michigan State University Extension",2020-12-23,The Food and Drug Administration has offered a...,A few examples include the offering of anti-a...,The Western Journal,"Health Scammers Prey on the Elderly, Here's Ho...",politics,https://www.westernjournal.com/health-scammers...
1,Aja Styles,2020-12-23,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...
2,Jake Johnson,2020-12-23,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...
3,Christine Favocci,2020-12-23,It is naive to think that either party is free...,The left has insisted that voter fraud is jus...,The Western Journal,PA Man Facing Charges of Unlawful Voting After...,tech,https://www.westernjournal.com/pa-man-facing-c...
4,Igor Derysh,2020-12-23,Fox aired segments debunking the claims after ...,Fox News and Fox Business just aired multiple...,Truthout,Fox News Forced to Debunk Its Own False Voting...,business,https://truthout.org/articles/fox-news-forced-...
...,...,...,...,...,...,...,...,...
272444,Vanguard,2020-12-05,The United States on Friday imposed new sancti...,US imposes visa restrictions on Chinese offic...,Vanguard News,US imposes visa restrictions on Chinese offici...,business,https://www.vanguardngr.com/2020/12/us-imposes...
272445,Guto Llewelyn,2020-12-05,Fan columnist Guto Llewelyn examines the situa...,When you subscribe we will use the informatio...,WalesOnline,The reality of Swansea City's striker concerns...,sport,https://www.walesonline.co.uk/sport/football/t...
272446,Laura McAllister,2020-12-05,"If successful, I’d be the first woman from the...",Former Wales international Laura McAllister h...,WalesOnline,"FIFA, football and Wales: Why I am standing fo...",sport,https://www.walesonline.co.uk/sport/sport-opin...
272447,Neil Lancefield PA,2020-12-05,One customer said a hotel was a 'filthy hovel'...,"Want the best food, film, music, arts and cul...",WalesOnline,Britannia ranked the UK's worst hotel chain fo...,tech,https://www.walesonline.co.uk/whats-on/whats-o...


#### Remove outlets / organisations from list of author names

In [9]:
remove = {'', 'Reuters', 'Reuters Staff', 'Associated', 'Vanguard', 'ABC News', 
          'The Washington Times http://www.washingtontimes.com', 'BWW News Desk', 'Press Trust of India', 
          'Press Association', 'PTI', 'Capital Market', 'PA Media', 'The Korea Herald', 'SA Transcripts',
          'Deutsche Welle (www.dw.com)', 'Associated Press', 'Afp', 'India.com News Desk', 'The Associated Press',
          'Australian Associated', 'FE Online', 'www.ETAuto.com', 'www.ETTelecom.com', 'F_488', 'Staff Writer',
          'India.com Sports Desk', 'www.ETEnergyworld.com', 'IANS', 'TV News Desk', 'ANI Press Release', 'AP', 
          'www.ETHealthworld.com', 'WRAL', 'India.com Entertainment Desk', 'ANI', 'HT Auto Desk', 
          'WND News Services', 'Stage Tube', 'Staff report', 'The Economist', 'AFP', 'F_300926', 
          'www.ETBrandEquity.com', 'FE Bureau', 'The Christian Science Monitor', 'Flanders News', 'BS Web Team',
          'India.com Lifestyle Staff', 'VietNamNet News', 'India.com Lifestyle Staff', 'India.com Viral News Desk',
          'Dailymail.com', 'The Moscow Times', 'www.ETCIO.com', 'www.ETCIO.com', 'SI Reporter', 
          'From Staff Reports','BWW', 'Entrepreneur en Español', 'Trefis Team', 'azfamily.com News Staff', 
          'From staff reports', 'Compiled by Democrat-Gazette Staff From Wire Reports', 
          'Classifieds Arizona Daily Star', '', 'Daily Express newspaper', 'www.ETRealty.com', 'Daily', 'Livemint',
          'Kyodo', 'India.com Business Desk', 'The Editorial Board', 'F_300833', 'HNN Staff','F_126', 'F_127',
          'India.com Education Desk', 'F_300824', 'Staff', 'doctor.ndtv.com', 'Daily News Editorial Board',
          'Expert Panel®', 'Staff Report', 'Entrepreneur Store',  'BWW Staff', 'to the editor', 'Dawn.com', 
          'WND Staff', 'Arizona Daily Star',  'F_300832', 'TV Scoop', None, 'STLNEWS', 
          'India.com Hindi News Desk', 'news18', "The Newspaper's Staff Reporter", 
          'Business Standard Editorial Comment', 'Business Standard', 'THE ASSOCIATED PRESS',  'Associated Press staff',
          'THE EDITORIAL STAFF', 'The Value Portfolio', 'Breakingviews columnists', 'StatBot | STLhighschoolSPORTS.com',
          'Team YS', 'F_200745', 'F_300923', 'Special to The Commercial', 'Tarot Astrologers',
          'Voice of the People', 'TNW Deals', 'Guardian community team', 'Baltimore Sun staff', 'News18',
          'Mail', 'AP Reporters', 'Avisol Capital Partners', 'Agencies', 'News staff', 'CNBC.com staff',
          'BroadwayWorld TV', 'The Conversation', 'F_200798', ' birminghammail.co.uk', 'ASSOCIATED PRESS',
          'F_200805', "The Age's View", 'BOOX Research', 'Associated Press Staff', 'Fun Trading', 'The Gazette',
          'American Heart Association News', 'Record Reporter', 'The Investment Doctor', 'Roadshow staff',
          'the Times Union Editorial Board', 'Baltimore Sun Editorial Board', 'The Editors of\xa0GQ', 
          'The Roanoke Times', 'The Spectator', 'Editorial staff', 'Individual Trader', 'Univision', None,
          'Arkansas Democrat-Gazette', 'Mirror.co.uk', 'ASSOCIATED PRESS STAFF', 'DerWesten - derwesten.de',
          'The New York Times', 'Wolf Report', 'The Gazette editorial board', 'Real Vision', 
          'Sun Sentinel Editorial Board', 'Voice of the Mirror', '@adwait', 'Expert Panel', 'The Buy Area Staff',
          'Volatility Surfer', 'Special To The Arizona Daily Star', 'NFL Staff', 'CNN Staff',
          'PJ Brown Special to the Arizona Daily Star', 'Invesco US', 'Bloomberg News', 'Quora', 'CNN Wire',
          'IP Banking Research', 'Daile Cross', 'The Washington Post', 'Bears of Wall Street', 'Teen Vogue Staff',
          'ICICI Securities', 'WIRED Staff', 'This Is', 'Cinemanía', 'STAFF EDITORIAL', 'StackCommerce', 'Axios',
         'Lincoln Journal Star', 'The Sunday Times', 'IE Online', 'Bloomberg', 'Associated Press Reporter' ,'Behance',
         'The Financial Express', 'Firstpost', '20minutos', 'Daily Herald report', 'Staff reports', 'MarketWatch',
         'Times Staff', 'Trapping Value', 'Q.ai - Investing Reimagined', 'CNET staff', 'YEC', 'Catholic Online', 
         'Stone Fox Capital', 'PA Sport Staff', 'Entrepreneur Staff', 'Keystone-SDA/ts', 'Associated Press reporters',
         "The Herald's View", 'Autoblog Staff', 'finanzen.net GmbH', 'Rolling Stone', 'AP | PTI', 'Story by Reuters',
         'PA reporters', 'Sports Hotline', 'Pioneer Press Staff', 'PA', 'Ticked Off!', 'Gazette readers', 'ValueZen',
         'India Knight', 'RMC SPORT', 'Truthout', 'MailOnline', 'Variety Staff', 'News Service of Florida', 'From wire reports',
         'Elephant Analytics', 'Chowhound Staff', 'Quad 7 Capital', 'The Associated Press, Associated Press',
         'undefined undefined', 'Orlando Sentinel Editorial Board', 'Clarín.com', 'Staff and wire reports', 'IndiaToday.in',
         'East Kilbride News', 'Retirement Pot', 'QuandaryFX', 'Perthshire Advertiser', 'The Inquirer Editorial Board',
         'CondÃ© Nast', 'WRITE TO LETTERS@THETIMES.CO.UK', 'Entrepreneur Middle East Staff', 'Editorial', 'WWD Staff',
         'STATE JOURNAL WIRE SERVICES', 'Think Change India', 'Shock Exchange', 'News4Jax staff', 'CNN staff',
         'The A.V. Club', 'Journal Star editorial board', 'Record View', 'Renaissance Capital IPO Research', 
          'MarketWatch Automation', 'Morning Call Staff', 'Pa Sport Staff', 'The Value Investor', 'Casa Editorial El Tiempo',
         'Hamilton Advertiser', 'HELLO! Canada', 'The Virginian-Pilot & Daily Press Editorial Board', 'Wishaw Press',
          'By CNN Underscored Staff', 'Political Comments', 'Mott Capital Management', 'Captain Alpha', 'STATE JOURNAL STAFF',
          'By <a href="/profiles/meg-wagner">Meg Wagner</a> and <a href="/profiles/melissa-mahtani">Melissa Mahtani</a>, CNN',
          'Franklin Templeton Investments', 'Ellevate', 'Pa', 'PA Reporters', 'Chicago Sun-Times', 'Navbharat Times', 'Long Player', 
          'By <a href="/profiles/meg-wagner">Meg Wagner</a>, Melissa Macaya, <a href="/profiles/melissa-mahtani">Melissa Mahtani</a>, <a href="/profiles/veronica-rocha">Veronica Rocha</a> and <a href="/profiles/amanda-wills">Amanda Wills</a>, CNN',
          'By <a href="/profiles/meg-wagner">Meg Wagner</a>, Melissa Macaya and Mike Hayes, CNN', 'SAPO', 'Slate Staff',
          'By <a href="/profiles/meg-wagner">Meg Wagner</a>, <a href="/profiles/fernando-alfonso-iii">Fernando Alfonso III</a>, Melissa Macaya, <a href="/profiles/melissa-mahtani">Melissa Mahtani</a>, <a href="/profiles/veronica-rocha">Veronica Rocha</a> and <a href="/profiles/amanda-wills">Amanda Wills</a>, CNN', 
          'By <a href="/profiles/donie-osullivan">Donie ' + "O'Sullivan</a>, " + '<a href="/profiles/kaya-yurieff">Kaya Yurieff</a>, Kelly Bourdet, the CNN Business team and contributors from across CNN', 
          'Japan Today', 'BrandWagon Online', 'Condé Nast', 'Gen Alpha', 'Chicago Tribune staff', 'Field Level Media', 
         'Tribune Content Agency', 'Opal Investment Research', 'Steerpike', 'The Value Pendulum', 'Sheen Bay Research',
         'SAP Guest', 'Guardian sport', 'Value Kicker', 'Staff Reports', 'Dividend Seeker', 'Wall Street Breakfast',
         'Stackcommerce', 'Keystone-SDA/ac', 'Compiled by Democrat-Gazette staff from wire reports', 'Markit', 'WisdomTree',
         'Carroll County Times Editorial Board', 'Hoya Capital Real Estate', 'NWA Democrat-Gazette'}

In [10]:
# remove other unicodes
for i in range(len(articles)):
    try:
        if not ''.join(filter(lambda character:ord(character) < 0x3000, articles.iloc[i].author)):
            remove.add(articles.iloc[i].author)
    except:
        pass
        
articles = articles[~articles['author'].isin(remove)]

In [11]:
articles

,author,date,description,full_text,site_name,title,topic,url
0,"Liz Josaitis, Michigan State University Extension",2020-12-23,The Food and Drug Administration has offered a...,A few examples include the offering of anti-a...,The Western Journal,"Health Scammers Prey on the Elderly, Here's Ho...",politics,https://www.westernjournal.com/health-scammers...
1,Aja Styles,2020-12-23,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...
2,Jake Johnson,2020-12-23,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...
3,Christine Favocci,2020-12-23,It is naive to think that either party is free...,The left has insisted that voter fraud is jus...,The Western Journal,PA Man Facing Charges of Unlawful Voting After...,tech,https://www.westernjournal.com/pa-man-facing-c...
4,Igor Derysh,2020-12-23,Fox aired segments debunking the claims after ...,Fox News and Fox Business just aired multiple...,Truthout,Fox News Forced to Debunk Its Own False Voting...,business,https://truthout.org/articles/fox-news-forced-...
...,...,...,...,...,...,...,...,...
272442,Nisha Mal,2020-12-05,"'It's all one big conundrum,' says Sheila Herbert",Woman's home is in Tier 2 while her garden fa...,WalesOnline,Woman's home is in Tier 2 while her garden fal...,tech,https://www.walesonline.co.uk/news/uk-news/wom...
272445,Guto Llewelyn,2020-12-05,Fan columnist Guto Llewelyn examines the situa...,When you subscribe we will use the informatio...,WalesOnline,The reality of Swansea City's striker concerns...,sport,https://www.walesonline.co.uk/sport/football/t...
272446,Laura McAllister,2020-12-05,"If successful, I’d be the first woman from the...",Former Wales international Laura McAllister h...,WalesOnline,"FIFA, football and Wales: Why I am standing fo...",sport,https://www.walesonline.co.uk/sport/sport-opin...
272447,Neil Lancefield PA,2020-12-05,One customer said a hotel was a 'filthy hovel'...,"Want the best food, film, music, arts and cul...",WalesOnline,Britannia ranked the UK's worst hotel chain fo...,tech,https://www.walesonline.co.uk/whats-on/whats-o...


#### Remove journalists who have authored less than 10 articles

In [12]:
articles['author_article_count'] = articles.groupby('author')['author'].transform('count')
articles = articles[articles.author_article_count >= 10]

<ipython-input-12-84b5e04ac8fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles['author_article_count'] = articles.groupby('author')['author'].transform('count')


In [13]:
articles = articles.reset_index(drop = True)

In [14]:
articles

,author,date,description,full_text,site_name,title,topic,url,author_article_count
0,Aja Styles,2020-12-23,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...,14
1,Jake Johnson,2020-12-23,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...,33
2,Christine Favocci,2020-12-23,It is naive to think that either party is free...,The left has insisted that voter fraud is jus...,The Western Journal,PA Man Facing Charges of Unlawful Voting After...,tech,https://www.westernjournal.com/pa-man-facing-c...,19
3,William Rivers Pitt,2020-12-23,What Trump may do in his waning days is only u...,"The endgame being played out by Donald Trump,...",Truthout,What Will Trump Attempt in His Last Days? We M...,politics,https://truthout.org/articles/what-will-trump-...,14
4,Amy Goodman,2020-12-23,Critics say the $900 billion relief package do...,As Congress passes a $900 billion coronavirus...,Truthout,The Insufficient COVID Stimulus Must Not Be Fo...,business,https://truthout.org/video/the-insufficient-co...,19
...,...,...,...,...,...,...,...,...,...
119957,Daniel Smith,2020-12-05,Ashley Towne decided to get in shape to marry ...,When you subscribe we will use the informatio...,WalesOnline,'I can't get married looking like this' Rower ...,sport,https://www.walesonline.co.uk/news/uk-news/i-c...,147
119958,Victoria Jones,2020-12-05,Experimenting on the ISS allows scientists to ...,When you subscribe we will use the informatio...,WalesOnline,Space experiment could unlock resources for mi...,tech,https://www.walesonline.co.uk/news/uk-news/spa...,92
119959,Nisha Mal,2020-12-05,All of the properties are Grade I listed build...,Three buildings featured in a Jane Austen nov...,WalesOnline,Three buildings featured in a Jane Austen nove...,tech,https://www.walesonline.co.uk/news/uk-news/thr...,66
119960,Nisha Mal,2020-12-05,"'It's all one big conundrum,' says Sheila Herbert",Woman's home is in Tier 2 while her garden fa...,WalesOnline,Woman's home is in Tier 2 while her garden fal...,tech,https://www.walesonline.co.uk/news/uk-news/wom...,66


#### Remove irrelevant keywords from name

In [15]:
def format_name(name):
    clean_name = name.lower()
    clean_name = re.sub(r'\|.*$', '', clean_name) # remove everything after |
    clean_name = re.sub(r'\,.*$', '', clean_name) # remove evrything after ,
    punctuations = '''!()[]{};:'"\,<>./?#$%^&*_~'''
    
    for x in clean_name.lower(): 
        if x in punctuations: clean_name = clean_name.replace(x, "") # remove punctuations except hyphen
    clean_name = re.sub(r'(\d+)', '', clean_name) # remove numbers
    clean_name = re.sub(r'(--)', '', clean_name)
    clean_name = ' '.join([i.strip() for i in clean_name.split()]) # remove spaces and lowercase
    clean_name = re.sub(r'(\w*@\w*)', '', clean_name) # remove emails
        
    clean_name = re.sub(r'(sa)(\s+)(\w+)(\s+)', '', clean_name) # SA editors
    clean_name = re.sub(r'(tulsa)(\s+)(world)', '', clean_name) # tulsa world editors
    clean_name = re.sub(r'(world-herald)(\s*)(\w*)(\s*)(\w*)', '', clean_name) # world herald editors
    clean_name = re.sub(r'(news)(\s+)(editor)', '', clean_name) # news editors
    clean_name = re.sub(r'(richmond)(\s+)(times-dispatch)', '', clean_name) # richmond editors
    clean_name = re.sub(r'(new)(\s+)(hampshire)(\s+)(union)(\s+)(leader)', '', clean_name) # new hampshire editors
    clean_name = re.sub(r'(for)(\s+)(the)(\s+)(state)(\s+)(journal)', '', clean_name) # state journal editors
    clean_name = re.sub(r'(union)(\s+)(leader)(\s+)(correspondent)', '', clean_name) # union leader correspondants
    clean_name = re.sub(r'(sfgate)', '', clean_name) # sfgate correspondants
    clean_name = re.sub(r'(special)(\s+)(from)(\s+)(the)(\s+)(gazette)', '', clean_name) # the gazette correspondants
    clean_name = re.sub(r'(times)(\s+)(correspondent)', '', clean_name) # times correspondants
    
    keywords = ['sa', 'tulsa', 'editor', 'writer', 'world-herald', 'news', 'world', 'richmond', 'times-dispatch', 'new',
                   'hampshire', 'union', 'leader', 'for', 'the', 'state', 'journal', 'correspondant', 'sfgate', 'special',
                   'from', 'the', 'gazette', 'times', 'staff', 'senior', 'dr', 'correspondent', 'by', 'editorial board',
                   'research', 'wire reports', 'security', 'real', 'estate', 'to', 'post', 'and', 'courier', 'policy',
                   'commercial', 'bureau', 'political', 'roanoke', 'college', 'football', 'editorial','democrat-gazette',
                   'arizona', 'daily', 'star', '--hamburg', 'column', 'lincoln', 'managing', 'backstage', 'with', 'sports',
                   'ii', 'iii', 'capitol', 'media', 'services']
    clean_name = ' '.join([i for i in clean_name.split() if i not in keywords]) # remove other keywords
    clean_name = re.sub(r'(@)', '', clean_name) # remove handles

    return clean_name.title()

In [16]:
articles['author_name_clean'] = articles.apply(lambda x: format_name(x['author']), axis=1)

In [17]:
articles

,author,date,description,full_text,site_name,title,topic,url,author_article_count,author_name_clean
0,Aja Styles,2020-12-23,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...,14,Aja Styles
1,Jake Johnson,2020-12-23,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...,33,Jake Johnson
2,Christine Favocci,2020-12-23,It is naive to think that either party is free...,The left has insisted that voter fraud is jus...,The Western Journal,PA Man Facing Charges of Unlawful Voting After...,tech,https://www.westernjournal.com/pa-man-facing-c...,19,Christine Favocci
3,William Rivers Pitt,2020-12-23,What Trump may do in his waning days is only u...,"The endgame being played out by Donald Trump,...",Truthout,What Will Trump Attempt in His Last Days? We M...,politics,https://truthout.org/articles/what-will-trump-...,14,William Rivers Pitt
4,Amy Goodman,2020-12-23,Critics say the $900 billion relief package do...,As Congress passes a $900 billion coronavirus...,Truthout,The Insufficient COVID Stimulus Must Not Be Fo...,business,https://truthout.org/video/the-insufficient-co...,19,Amy Goodman
...,...,...,...,...,...,...,...,...,...,...
119957,Daniel Smith,2020-12-05,Ashley Towne decided to get in shape to marry ...,When you subscribe we will use the informatio...,WalesOnline,'I can't get married looking like this' Rower ...,sport,https://www.walesonline.co.uk/news/uk-news/i-c...,147,Daniel Smith
119958,Victoria Jones,2020-12-05,Experimenting on the ISS allows scientists to ...,When you subscribe we will use the informatio...,WalesOnline,Space experiment could unlock resources for mi...,tech,https://www.walesonline.co.uk/news/uk-news/spa...,92,Victoria Jones
119959,Nisha Mal,2020-12-05,All of the properties are Grade I listed build...,Three buildings featured in a Jane Austen nov...,WalesOnline,Three buildings featured in a Jane Austen nove...,tech,https://www.walesonline.co.uk/news/uk-news/thr...,66,Nisha Mal
119960,Nisha Mal,2020-12-05,"'It's all one big conundrum,' says Sheila Herbert",Woman's home is in Tier 2 while her garden fa...,WalesOnline,Woman's home is in Tier 2 while her garden fal...,tech,https://www.walesonline.co.uk/news/uk-news/wom...,66,Nisha Mal


In [18]:
journalists = articles.copy(deep=True)[['author_name_clean', 'author_article_count']].drop_duplicates().reset_index(drop = True)

In [19]:
journalists

,author_name_clean,author_article_count
0,Aja Styles,14
1,Jake Johnson,33
2,Christine Favocci,19
3,William Rivers Pitt,14
4,Amy Goodman,19
...,...,...
3831,Ceimin Burke,15
3832,Harrison Schwartz,10
3833,Amanda Hancock,11
3834,Lisa Mcloughlin,10


### Gathering Muckrack URLs

#### Creating raw URLs

In [20]:
def create_url(name):
    url = 'https://muckrack.com/' + name.lower().replace(' ', '-').strip('-') # make URL from author name
    return url

journalists['request_url'] = journalists.apply(lambda x: create_url(x['author_name_clean']), axis=1)
journalists['redirect_url'] = journalists.apply(lambda x: create_url(x['author_name_clean']), axis=1)

In [21]:
journalists

,author_name_clean,author_article_count,request_url,redirect_url
0,Aja Styles,14,https://muckrack.com/aja-styles,https://muckrack.com/aja-styles
1,Jake Johnson,33,https://muckrack.com/jake-johnson,https://muckrack.com/jake-johnson
2,Christine Favocci,19,https://muckrack.com/christine-favocci,https://muckrack.com/christine-favocci
3,William Rivers Pitt,14,https://muckrack.com/william-rivers-pitt,https://muckrack.com/william-rivers-pitt
4,Amy Goodman,19,https://muckrack.com/amy-goodman,https://muckrack.com/amy-goodman
...,...,...,...,...
3831,Ceimin Burke,15,https://muckrack.com/ceimin-burke,https://muckrack.com/ceimin-burke
3832,Harrison Schwartz,10,https://muckrack.com/harrison-schwartz,https://muckrack.com/harrison-schwartz
3833,Amanda Hancock,11,https://muckrack.com/amanda-hancock,https://muckrack.com/amanda-hancock
3834,Lisa Mcloughlin,10,https://muckrack.com/lisa-mcloughlin,https://muckrack.com/lisa-mcloughlin


In [22]:
# add extra columns
for j in range(len(journalists)):
    journalists.loc[j, 'method'] = '-'
    journalists.loc[j, 'confidence'] = '-'

In [23]:
ua = UserAgent()

#### Experimenting with TF IDF Matching vs Levenshtein distance

In [24]:
lookup = list(muckrack_sitemap.url)
k_matches = 1
ngram_length = 3

In [25]:
def match_with_tfidf(wrong_url):
    wrong_url = [wrong_url]
    results = tm.matcher(wrong_url, lookup, k_matches, ngram_length)
    match_confidence = results['Match Confidence'].iloc[0]
    right_url = results['Lookup 1'].iloc[0]
    return right_url, match_confidence

In [26]:
def match_with_levenshtein(wrong_url):
    try:
        highest = process.extractOne(wrong_url, list(muckrack_sitemap.url))
        return highest
    except:
        return None

In [27]:
print("Computing 1 string match")

start = time.time()
for i in range(1):
    match_with_tfidf(journalists.iloc[i].request_url)
end = time.time()
delta = end - start
print("(TF-IDF) Seconds elapsed", delta)

start = time.time()
for i in range(1):
    match_with_levenshtein(journalists.iloc[i].request_url)
end = time.time()
delta = end - start
print("(Levenshtein) Seconds elapsed", delta)

Computing 1 string match
(TF-IDF) Seconds elapsed 4.954720735549927
(Levenshtein) Seconds elapsed 71.20504713058472


In [28]:
print("Computing 10 string matches")

start = time.time()
for i in range(10):
    match_with_tfidf(journalists.iloc[i].request_url)
end = time.time()
delta = end - start
print("(TF-IDF) Seconds elapsed", delta)

start = time.time()
for i in range(10):
    match_with_levenshtein(journalists.iloc[i].request_url)
end = time.time()
delta = end - start
print("(Levenshtein) Seconds elapsed", delta)

Computing 10 string matches
(TF-IDF) Seconds elapsed 54.540432929992676
(Levenshtein) Seconds elapsed 811.230033159256


In [29]:
print("Computing 100 string matches")

start = time.time()
for i in range(100):
    match_with_tfidf(journalists.iloc[i].request_url)
end = time.time()
delta = end - start
print("(TF-IDF) Seconds elapsed", delta)

start = time.time()
for i in range(100):
    match_with_levenshtein(journalists.iloc[i].request_url)
end = time.time()
delta = end - start
print("(Levenshtein) Seconds elapsed", delta)

Computing 100 string matches
(TF-IDF) Seconds elapsed 589.8373725414276
(Levenshtein) Seconds elapsed 9547.315568685532


#### Requesting URLs with Muckrack + TF IDF

In [32]:
journalists

,author_name_clean,author_article_count,request_url,redirect_url,method,confidence
0,Aja Styles,14,https://muckrack.com/aja-styles,https://muckrack.com/aja-styles,-,-
1,Jake Johnson,33,https://muckrack.com/jake-johnson,https://muckrack.com/jake-johnson,-,-
2,Christine Favocci,19,https://muckrack.com/christine-favocci,https://muckrack.com/christine-favocci,-,-
3,William Rivers Pitt,14,https://muckrack.com/william-rivers-pitt,https://muckrack.com/william-rivers-pitt,-,-
4,Amy Goodman,19,https://muckrack.com/amy-goodman,https://muckrack.com/amy-goodman,-,-
...,...,...,...,...,...,...
3831,Ceimin Burke,15,https://muckrack.com/ceimin-burke,https://muckrack.com/ceimin-burke,-,-
3832,Harrison Schwartz,10,https://muckrack.com/harrison-schwartz,https://muckrack.com/harrison-schwartz,-,-
3833,Amanda Hancock,11,https://muckrack.com/amanda-hancock,https://muckrack.com/amanda-hancock,-,-
3834,Lisa Mcloughlin,10,https://muckrack.com/lisa-mcloughlin,https://muckrack.com/lisa-mcloughlin,-,-


In [33]:
for j in range(len(journalists)):
    url = journalists.iloc[j].request_url
    # print(j)
    try:
        req = Request(url, headers = {'User-Agent': ua.random})
        res = urlopen(req)
        sauce = res.read()
        journalists.loc[j, 'method'] = 'brute force'
        journalists.loc[j, 'redirect_url'] = res.url
        # print(j, "found brute force", url)
    except:
        url, confidence = match_with_tfidf(url)
        try:
            req = Request(url, headers = {'User-Agent': ua.random})
            res = urlopen(req)
            sauce = res.read()
            journalists.loc[j, 'method'] = 'tfidf match'
            journalists.loc[j, 'confidence'] = confidence
            journalists.loc[j, 'request_url'] = url
            journalists.loc[j, 'redirect_url'] = res.url
            # print(j, "found tfidf", url)
        except:
            pass
            # print(j, "found nothing", url)
            # journalists.loc[j, 'method'] = '-'
            # google scraper

#### Web search for remaining URLs

In [40]:
client = Algorithmia.client('simYm8DHcOm5sOaSIaAzH/Xzbin1')
algo = client.algo('specrom/Google_scraper/0.1.4')
algo.set_options(timeout=100) # optional

In [46]:
for i in range(len(journalists)):
    if (journalists.iloc[i].method == 'tfidf match' and float(journalists.iloc[i].confidence) < 0.85) or (journalists.iloc[i].method == '-') or (journalists.iloc[i].method == 'no match'):
        query = {"query": f"{journalists.iloc[i].author_name_clean} muckrack"}
        results = algo.pipe(query).result
        if results and 'muckrack' in results[0]['url']:
            journalists.at[i, 'request_url'] = results[0]['url']
            journalists.at[i, 'redirect_url'] = results[0]['url']
            journalists.at[i, 'method'] = 'google scrape'
            journalists.at[i, 'confidence'] = '-'    
        else:
            journalists.at[i, 'method'] = '-'

In [47]:
journalists.method.value_counts()

brute force      3482
google scrape     237
tfidf match       108
-                   9
Name: method, dtype: int64

In [49]:
journalists = journalists[journalists.method != '-'].reset_index(drop = True)

### Extract additional information

In [51]:
def extract_data(soup):
    
    beats, title, location, links = '-', '-', '-', '-'
    try:
        if soup.find(class_ = 'person-details-item person-details-beats'):
            beats = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-tasks icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    except:
        pass
    
    try:
        if soup.find(class_ = 'person-details-item person-details-title'):
            title = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-building icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    except:
        pass
    
    try:
        if soup.find(class_ = 'person-details-item person-details-location'):
            location = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-map-marker-alt icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    except:
        pass
    
    try: 
        if soup.find(class_ = 'profile-section-social'):
            urls = []
            for i in soup.find(class_ = 'profile-section-social'):
                temp = [j[5:] for j in str(i).split() if 'href' in j]
                if temp: 
                    temp = temp[0].strip('"')
                    urls.append(temp)
            links = ', '.join(urls)
    except:
        pass
    
    return beats, title, location, links

In [53]:
for i in range(len(journalists)):
    # print(i)
    muckrack = journalists.iloc[i].redirect_url
    req = Request(muckrack, headers = {'User-Agent': ua.random})
    try:
        res = urlopen(req)
        sauce = res.read()
        soup = bs.BeautifulSoup(sauce,'lxml')
        beats, title, location, links = extract_data(soup)
        journalists.loc[i, 'beats'], journalists.loc[i, 'title'], journalists.loc[i, 'location'], journalists.loc[i, 'links'] = beats, title, location, links
    except:
        journalists.loc[i, 'beats'], journalists.loc[i, 'title'], journalists.loc[i, 'location'], journalists.loc[i, 'links'] = '-', '-', '-', '-'
    # print(muckrack, extract_data(soup))

In [55]:
articles.dropna(subset = ['author_name_clean']).reset_index(drop = True).to_csv('../Data/articles_v1.csv', index = False)

In [57]:
journalists.dropna().reset_index(drop = True).to_csv('../Data/journalists_v1.csv', index = False)